In [1]:
import os
import sys

import pandas as pd
from matplotlib import pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score

import tqdm
import numpy as np
import random

import h2o
from h2o.automl import H2OAutoML

import warnings
warnings.filterwarnings("ignore")

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 17.0.5+9-LTS-191, mixed mode, sharing)
  Starting server from D:\Python312\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\ghsqj\AppData\Local\Temp\tmpcpe5tyt8
  JVM stdout: C:\Users\ghsqj\AppData\Local\Temp\tmpcpe5tyt8\h2o_Hong_Beomsun_started_from_python.out
  JVM stderr: C:\Users\ghsqj\AppData\Local\Temp\tmpcpe5tyt8\h2o_Hong_Beomsun_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Asia/Seoul
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.4
H2O_cluster_version_age:,19 days
H2O_cluster_name:,H2O_from_python_Hong_Beomsun_h2u2q1
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.984 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [3]:
class Config:
    ROOT_DIR = './data/huge'
    SEED = 42

CONFIG = Config()

In [4]:
seed = CONFIG.SEED
np.random.seed(seed)
random.seed(seed)

In [5]:
train_df = pd.read_csv(os.path.join(CONFIG.ROOT_DIR, "train_df.csv"), index_col='Serial Number')
test_x = pd.read_csv(os.path.join(CONFIG.ROOT_DIR, "test_x.csv"), index_col='Serial Number')
train_y = pd.read_csv(os.path.join(CONFIG.ROOT_DIR, "train_y.csv"), index_col='Serial Number')

In [6]:
train_df['TIMESTAMP'] = pd.to_datetime(train_df['TIMESTAMP'])
train_df['TIMESTAMP'] = train_df['TIMESTAMP'].map(lambda t: t.strftime('%Y-%m-%d %H:%M'))

test_x['TIMESTAMP'] = pd.to_datetime(test_x['TIMESTAMP'])
test_x['TIMESTAMP'] = test_x['TIMESTAMP'].map(lambda t: t.strftime('%Y-%m-%d %H:%M'))

serial_key = list(train_df.columns)[0]
date_time_key = list(train_df.columns)[1]
feature_keys = list(train_df.columns)[3:-1]
target_key = list(train_df.columns)[-1]

train_x = train_df.drop(columns='Y')

In [7]:
print("Train Data 크기 :", train_x.shape)
print("Test Data 크기 :", test_x.shape)

print("Train Data의 Serial Number의 unique 값 :", len(train_x[serial_key].unique()))
print("Test Data의 Serial Number의 unique 값 :", len(test_x[serial_key].unique()))

Train Data 크기 : (555456, 19)
Test Data 크기 : (138880, 19)
Train Data의 Serial Number의 unique 값 : 97860
Test Data의 Serial Number의 unique 값 : 72562


In [8]:
train_x.head(3)

,TIMESTAMP,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18
Serial Number,,,,,,,,,,,,,,,,,,,
19,2020-02-09 16:24,EI83N072710203N8H,22,18,22,16,32,99,91219,43095,2132,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19,2020-02-10 16:39,EI83N072710203N8H,22,18,22,16,32,99,91413,43214,2133,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19,2020-02-11 16:54,EI83N072710203N8H,22,18,22,16,32,99,91606,43332,2135,49.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


### PreProcessing

In [9]:
train_x.drop(columns='X1', inplace=True)
test_x.drop(columns='X1', inplace=True)

In [10]:
scaler = StandardScaler()

train_x[feature_keys] = scaler.fit_transform(train_df[feature_keys])
test_x[feature_keys] = scaler.transform(test_x[feature_keys])

In [11]:
train_x_by_serial = [group[1] for group in train_x.groupby(train_x.index)]
test_x_by_serial = [group[1] for group in test_x.groupby(test_x.index)]

train_x_by_serial = [group.sort_values('TIMESTAMP') for group in train_x_by_serial]
test_x_by_serial = [group.sort_values('TIMESTAMP') for group in test_x_by_serial]

### Dataset

In [12]:
def train_test_split(Xs, ys, test_ratio=0.2):
    data_per_label = {}

    for x, y in zip(Xs, ys):
        label = y
        if label not in data_per_label:
            data_per_label[label] = []
        data_per_label[label].append((x, y))

    train = []
    test = []

    for label in data_per_label:
        data = data_per_label[label]
        n_test = int(len(data) * test_ratio)
        test += data[:n_test]
        train += data[n_test:]

    X_train, y_train = zip(*train)
    X_test, y_test = zip(*test)

    return X_train, X_test, y_train, y_test

In [13]:
X_train, X_val, y_train, y_val = train_test_split(train_x_by_serial, train_y['Y'], test_ratio=0.2)

X_train = [x.drop(columns='TIMESTAMP') for x in X_train]
X_val = [x.drop(columns='TIMESTAMP') for x in X_val]
X_test = [x.drop(columns='TIMESTAMP') for x in test_x_by_serial]

print("Train Data의 개수 :", len(X_train))
print("Validation Data의 개수 :", len(X_val))
print("Test Data의 개수 :", len(X_test))

Train Data의 개수 : 6618
Validation Data의 개수 : 1654
Test Data의 개수 : 2069


In [14]:
def align_data(data, series_length):
    data_features = [x[feature_keys] for x in data]
    len_data = len(data_features)
    length_aligned_X = []
    for x in data_features:
        if len(x) >= series_length:
            length_aligned_X.append(x[-series_length:])
        else:
            length_aligned_X.append(x.append([x.iloc[-1]] * (series_length - len(x))))
    return np.array(length_aligned_X).reshape(len_data, -1)

In [15]:
series_length = 50

X_train = align_data(X_train, series_length)
X_val = align_data(X_val, series_length)
X_test = align_data(X_test, series_length)

In [16]:
y_train = np.array(y_train)
y_val = np.array(y_val)

In [17]:
train_h2o = h2o.H2OFrame(X_train)
train_h2o['Y'] = h2o.H2OFrame(y_train)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [19]:
train_h2o['Y'] = train_h2o['Y'].asfactor()

In [22]:
train_h2o

C1          C2         C3        C4          C5         C6          C7          C8         C9         C10         C11         C12         C13         C14         C15          C16        C17         C18        C19       C20         C21        C22         C23         C24        C25         C26         C27         C28         C29         C30         C31          C32        C33         C34        C35       C36         C37        C38          C39         C40        C41          C42         C43         C44         C45         C46         C47          C48        C49         C50        C51       C52         C53        C54          C55         C56        C57         C58         C59         C60         C61         C62         C63          C64        C65         C66        C67       C68         C69        C70          C71         C72        C73         C74         C75         C76         C77         C78         C79          C80        C81         C82        C83       C84         C85        C86          C87         C88        C89        C90         C91         C92        C93         C94         C95          C96        C97        C98        C99      C100        C101       C102         C103        C104       C105       C106        C107        C108       C109        C110        C111         C112       C113       C114       C115      C116        C117       C118         C119        C120       C121      C122        C123        C124      C125        C126        C127         C128       C129        C130       C131      C132        C133        C134        C135        C136       C137      C138        C139        C140      C141        C142        C143         C144       C145        C146       C147      C148        C149        C150         C151        C152       C153      C154        C155        C156      C157        C158        C159         C160       C161        C162       C163      C164        C165        C166         C167        C168       C169      C170        C171        C172      C173        C174        C175         C176       C177        C178       C179      C180        C181        C182        C183        C184       C185      C186        C187        C188      C189        C190        C191         C192       C193        C194       C195      C196        C197        C198        C199        C200       C201      C202        C203        C204      C205        C206        C207         C208       C209        C210       C211      C212        C213        C214        C215        C216       C217      C218        C219        C220      C221        C222        C223         C224       C225        C226       C227      C228        C229        C230        C231        C232        C233      C234        C235        C236      C237        C238        C239         C240       C241        C242       C243      C244        C245        C246        C247        C248        C249      C250        C251        C252      C253        C254        C255         C256       C257        C258       C259      C260        C261        C262        C263        C264        C265      C266        C267        C268      C269        C270        C271         C272       C273        C274       C275      C276        C277        C278        C279        C280        C281      C282        C283        C284      C285        C286        C287         C288       C289        C290       C291      C292        C293        C294        C295        C296        C297      C298        C299        C300     C301        C302        C303         C304       C305        C306       C307      C308        C309        C310        C311        C312       C313      C314        C315        C316      C317        C318        C319         C320       C321       C322       C323      C324        C325        C326        C327        C328       C329      C330        C331        C332      C333        C334        C335         C336       C337       C338       C339      C340        C341       C342        C343        C344       C345     C346        C

### Training

In [26]:
aml = H2OAutoML(max_models=None, seed=CONFIG.SEED, balance_classes=True, max_runtime_secs=6000)
aml.train(x = train_h2o.columns[:-1], y='Y', training_frame=train_h2o)

AutoML progress: |
02:45:33.473: AutoML: XGBoost is not available; skipping it.



███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_1_AutoML_4_20240729_24533


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    95                 95                          74859                  9            15           14.9263       26            72            57.9053

ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.0014043101971201633
RMSE: 0.03747412703613205
LogLoss: 0.013797284434089938
Mean Per-Class Error: 0.0
AUC: 1.0
AUCPR: 1.0
Gini: 1.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.714318779335739
       0     1     Error    Rate
-----  ----  ----  -------  -------------
0      5648  0     0        (0.0/5648.0)
1      0     5645  0        (0.0/5645.0)
Total  5648  5645  0        (0.0/11293.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.714319     1         292
max f2                       0.714319     1         292
max f0point5                 0.714319     1         292
max accuracy                 0.714319     1         292
max precision                0.999666     1         0
max recall                   0.714319     1         292
max specificity              0.999666     1         0
max absolute_mcc             0.714319     1         292
max min_per_class_accuracy   0.714319     1         292
max mean_per_class_accuracy  0.714319     1         292
max tns                      0.999666     5648      0
max fns                      0.999666     5609      0
max fps                      1.5272e-05   5648      399
max tps                      0.714319     5645      292
max tnr                      0.999666     1         0
max fnr                      0.999666     0.993623  0
max fpr                      1.5272e-05   1         399
max tpr                      0.714319     1         292

Gains/Lift Table: Avg response rate: 49.99 %, avg score: 48.78 %
group    cumulative_data_fraction    lower_threshold    lift     cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain     cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  -------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  -------  -----------------  --------------------
1        0.0100062                   0.99954            2.00053  2.00053            1                0.999609     1                           0.999609            0.0200177       0.0200177                  100.053  100.053            0.0200177
2        0.0201009                   0.999441           2.00053  2.00053            1                0.999489     1                           0.999549            0.0201949       0.0402126                  100.053  100.053            0.0402126
3        0.0301071                   0.999364           2.00053  2.00053            1                0.999403     1                           0.9995              0.0200177       0.0602303                  100.053  100.053            0.0602303
4        0.0404675                   0.999292           2.00053  2.00053            1                0.99933      1                           0.999457            0.0207263       0.0809566                  100.053  100.053            0.0809566
5        0.0502081                   0.999206           2.00053  2.00053            1                0.999251     1                           0.999417            0.0194863       0.100443                   100.

In [27]:
lb = aml.leaderboard
print(lb)

model_id                            auc    logloss     aucpr    mean_per_class_error      rmse        mse
GBM_1_AutoML_4_20240729_24533  0.996799  0.0525672  0.986459               0.0333965  0.116261  0.0135166
DRF_1_AutoML_4_20240729_24533  0.994626  0.094744   0.97664                0.0420295  0.163633  0.0267757
GLM_1_AutoML_4_20240729_24533  0.979586  0.143869   0.907362               0.104506   0.197472  0.0389952
[3 rows x 7 columns]



In [66]:
X_val_h2o = h2o.H2OFrame(X_val)
y_val_pred = aml.leader.predict(X_val_h2o)
y_val_pred = y_val_pred.as_data_frame()['predict'].values.flatten()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [67]:
f1 = f1_score(y_val, y_val_pred, average='macro')
print(f'검증 데이터에 대한 F1 Score: {f1:.8f}')

검증 데이터에 대한 F1 Score: 0.91830308


### Pred

In [ ]:
submission = pd.read_csv(os.path.join(CONFIG.ROOT_DIR, "test_y.csv"), index_col='Serial Number')

test_h2o = h2o.H2OFrame(X_test)

y_test_pred = clf.predict(X_test)
submission["Y"] = y_test_pred
submission.to_csv("submission.csv", index_label='Serial Number')

In [ ]:
submission = pd.read_csv("submission.csv", index_col='Serial Number')
submission

In [ ]:
print(submission["Y"].mean())
print(train_y.mean())

Public Score : 0.8130563798219584